In [7]:
def get(nums, i):
	if i == -1 or i == len(nums):
		return -float('inf')
	else:
		return nums[i]
def find_top(nums):
    if len(nums) == 0:
        return -1
    if len(nums) == 1:
        return nums[0]
    l, r = 0, len(nums) - 1
    while l <= r:
        mid = (l + r) // 2  # mid = l + (r – l) // 2
        if get(nums, mid - 1) < get(nums, mid) > get(nums, mid + 1):
            return mid
        if get(nums, mid - 1) > get(nums, mid):
            r = mid - 1
        elif get(nums, mid + 1) > get(nums, mid):
            l = mid + 1


In [5]:
nums = [3, 1, 2, 4]
find_top(nums)

0

In [7]:
from run_llama import LlamaDataset, create_data
import argparse
from tokenizer import Tokenizer
from torch.utils.data import DataLoader

def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument("--train", type=str, default="data/cfimdb-train.txt")
	parser.add_argument("--dev", type=str, default="data/cfimdb-dev.txt")
	parser.add_argument("--test", type=str, default="data/cfimdb-test.txt")
	parser.add_argument("--label-names", type=str, default="data/cfimdb-label-mapping.json")
	parser.add_argument("--pretrained-model-path", type=str, default="stories42M.pt")
	parser.add_argument("--max_sentence_len", type=int, default=None)
	parser.add_argument("--seed", type=int, default=1337)
	parser.add_argument("--epochs", type=int, default=5)
	parser.add_argument("--option", type=str,
						help='prompt: the Llama parameters are frozen; finetune: Llama parameters are updated',
						choices=('generate', 'prompt', 'finetune', 'lora'), default="generate")
	parser.add_argument("--use_gpu", action='store_true')
	parser.add_argument("--generated_sentence_low_temp_out", type=str, default="generated-sentence-temp-0.txt")
	parser.add_argument("--generated_sentence_high_temp_out", type=str, default="generated-sentence-temp-1.txt")
	parser.add_argument("--dev_out", type=str, default="cfimdb-dev-prompting-output.txt")
	parser.add_argument("--test_out", type=str, default="cfimdb-test-prompting-output.txt")
	parser.add_argument("--lora_rank", type=int, default=4, help="LoRA rank (r)")
	parser.add_argument("--lora_alpha", type=float, default=1.0, help="LoRA alpha scaling factor")

	# hyper parameters
	parser.add_argument("--batch_size", help='sst: 64, cfimdb: 8 can fit a 12GB GPU', type=int, default=8)
	parser.add_argument("--hidden_dropout_prob", type=float, default=0.3)
	parser.add_argument("--lr", type=float, help="learning rate, default lr for 'pretrain': 1e-3, 'finetune': 1e-5",
						default=2e-5)

	args, _ = parser.parse_known_args()
	print(f"args: {vars(args)}")
	return args
args = get_args()

tokenizer = Tokenizer(args.max_sentence_len)
train_data, num_labels = create_data(args.train, tokenizer, 'train')
train_dataset = LlamaDataset(train_data, args)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size,
								  collate_fn=train_dataset.collate_fn)
for step, batch in enumerate(train_dataloader):
    b_ids, b_labels, b_sents, b_padding_mask = batch['token_ids'], batch['labels'], batch['sents'], batch['padding_mask']
    print(b_padding_mask.sum(dim=1))
    break

args: {'train': 'data/cfimdb-train.txt', 'dev': 'data/cfimdb-dev.txt', 'test': 'data/cfimdb-test.txt', 'label_names': 'data/cfimdb-label-mapping.json', 'pretrained_model_path': 'stories42M.pt', 'max_sentence_len': None, 'seed': 1337, 'epochs': 5, 'option': 'generate', 'use_gpu': False, 'generated_sentence_low_temp_out': 'generated-sentence-temp-0.txt', 'generated_sentence_high_temp_out': 'generated-sentence-temp-1.txt', 'dev_out': 'cfimdb-dev-prompting-output.txt', 'test_out': 'cfimdb-test-prompting-output.txt', 'lora_rank': 4, 'lora_alpha': 1.0, 'batch_size': 8, 'hidden_dropout_prob': 0.3, 'lr': 2e-05}
load 1707 data from data/cfimdb-train.txt
tensor([192, 177, 237, 195, 390, 227, 172, 250])
